In [4]:
import sys
import numpy
import copy

In [4]:
# enumerate binary trees of size n
# the standard form equations are B = U + V, U = z, V = B * B
# intermediate values are tabulated by a dict k -> (b_k, u_k, v_k)

idxB = 0
idxU = 1
idxV = 2

def tabulate(k, table):
    v_k = 0
    if k != 0:
        for i in range(1,k):
            v_k += table[i][idxB] * table[k-i][idxB]
    u_k = 1 if k == 1 else 0
    b_k = u_k + v_k
    table[k] = (b_k, u_k, v_k)
    return table
    
def BinaryTree(n):
    t = {}
    for k in range(n+1):
        t = tabulate(k, t)
    print t[n][idxB]
    return

for k in range(0,20):
    BinaryTree(k)

0
1
1
2
5
14
42
132
429
1430
4862
16796
58786
208012
742900
2674440
9694845
35357670
129644790
477638700


In [11]:
# enumerate combinatorial structures from standard form
# supports union, cartesian product and terminal rules
# intermediate values are tabulated by a dict k -> (dict r -> r_k)

class Union:
    def __init__(self, *args): # args[0] = args[1] + args[2]
        self.Type = 'Union'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Union) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)

class Product:
    def __init__(self, *args): # args[0] = args[1] * args[2]
        self.Type = 'Product'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Product) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)

class Terminal:
    def __init__(self, *args):
        self.Type = 'Terminal'
        if len(args) == 2:
            self.Value = args[0]
            self.Size = args[1]
        elif len(args) == 1:
            self.Size = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Terminal) and self.Size == other.Size

# note: a Cartesian product rule cannot have valuation 0
def evaluateRule(rule, k, table):
    if k not in table:
        table[k] = {}
    if k == 0:
        table[k][rule.Value] = 0
    elif isinstance(rule, Union):
        table[k][rule.Value] = table[k][rule.SubRule1] + table[k][rule.SubRule2]
    elif isinstance(rule, Product):
        table[k][rule.Value] = 0
        for i in range(1,k):
            table[k][rule.Value] += table[i][rule.SubRule1] * table[k-i][rule.SubRule2]
    elif isinstance(rule, Terminal):
        table[k][rule.Value] = 1 if rule.Size == k else 0
    else: raise Exception('Unsupported rule')
    return table

# compute the valuations of a list of rules
# implementation of algorithm on page 28, section 1.4
def computeRuleValuations(rules):
    val = {}
    for r in rules:
        if isinstance(r, Terminal):
            val[r.Value] = r.Size
        else:
            val[r.Value] = sys.maxint
    return valuate(val, rules)

# current method is to sort by decreasing valuation
# and sort all terminal rules to the front of the list
def sortRulesByValuation(rules):
    d = computeRuleValuations(rules).items()
    d_sorted = sorted(d, key=lambda x:x[1], reverse=True)
    r_sorted = []
    for (k,v) in d_sorted:
        for r in rules:
            if r.Value == k:
                if isinstance(r, Terminal):
                    r_sorted = [r] + r_sorted
                else:
                    r_sorted.append(r)
    return r_sorted
    
# helper for computeRuleValuations()
def valuate(v, rules):
    done = True
    for r in rules:
        prev = v[r.Value]
        if isinstance(r, Terminal):
            continue
        elif isinstance(r, Union):
            v[r.Value] = min(v[r.SubRule1], v[r.SubRule2])
        elif isinstance(r, Product):
            v[r.Value] = v[r.SubRule1] + v[r.SubRule2]
        else: raise Exception('Unsupported rule')
        if v[r.Value] != prev: done = False
    if done: return v
    else: return valuate(v, rules)

# rules should be sorted in order of increasing minimality
def EnumerateFromStandardForm(rules, n):
    t = {}
    for k in range(n+1):
        for r in rules:
            t = evaluateRule(r, k, t)
    return t

# test on a binary tree
def BTree(n):
    rules = [Union('B', 'U', 'V'), Product('V', 'B', 'B'), Terminal('U', 1)]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['B']

# test on a unary-binary tree
def UTree(n):
    rules = [Union('U', 'A', 'B'), Terminal('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['U']

# test on a symbolic equation
def SymbolicEquation(eq, n):
    rules = ConvertToStandardForm(eq)
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    seq = []
    for i in range(n):
        seq += [tab[i][eq.Value]]
    return seq
    
# runable code goes here
op = Union('X', Terminal(1), Union(Product(Terminal(1), 'X'), \
    Product(Product(Terminal(1), 'X'), 'X')))
# op = Union('X', Terminal(1), Product('X', 'X'))
print SymbolicEquation(op, 20)

[0, 1, 1, 2, 4, 9, 21, 51, 127, 323, 835, 2188, 5798, 15511, 41835, 113634, 310572, 853467, 2356779, 6536382]


In [5]:
# convert a symbolic equation to standard form
# example: B = Z + B * B -> [B = U + V, U = Z, V = B * B]

# UPDATE - FIXED - equality function in Terminal class was not
# defined properly (but still need to test on more complex
# symbolic equations)
#
# only one problem at the moment - may produce redundant equations
# e.g. - C = A + B, ..., E = A + B
# this does not effect correctness but may effect performance
# I believe this can be fixed with terminal substitutions but haven't
# managed to successfully implement that yet (tricky!)

def ConvertToStandardForm(eq):
    r, _ = convert(eq, {}, 65)
    return r.values()

# helper function for ConvertToStandardForm
def convert(op, rules, v):
    op1, op2 = (op.SubRule1, op.SubRule2)
    val1, val2 = (None, None)
    evalLeft, evalRight = (False, False)
    if isinstance(op1, Product) or isinstance(op1, Union):
        for r in rules.values():
            rSub = copy.deepcopy(r)
            if isinstance(rSub, Union) or isinstance(rSub, Product):
                if rSub.SubRule1 in rules:
                    if isinstance(rules[rSub.SubRule1], Terminal):
                        rSub.SubRule1 = rules[rSub.SubRule1]
                if rSub.SubRule2 in rules:
                    if isinstance(rules[rSub.SubRule2], Terminal):
                        rSub.SubRule2 = rules[rSub.SubRule2]
            if rSub == op1:
                val1 = r.Value
                break
        if val1 == None: 
            val1 = chr(v)
            v += 1
            evalLeft = True
    elif isinstance(op1, Terminal):
        for r in rules.values():
            if r.Size == op1.Size:
                val1 = r.Value
                break
        if val1 == None:
            val1 = chr(v)
            v += 1
            op1.Value = val1
            rules[val1] = op1
    else: val1 = op1
    if isinstance(op2, Product) or isinstance(op2, Union):
        for r in rules.values():
            rSub = copy.deepcopy(r)
            if isinstance(rSub, Union) or isinstance(rSub, Product):
                if rSub.SubRule1 in rules:
                    if isinstance(rules[rSub.SubRule1], Terminal):
                        rSub.SubRule1 = rules[rSub.SubRule1]
                if rSub.SubRule2 in rules:
                    if isinstance(rules[rSub.SubRule2], Terminal):
                        rSub.SubRule2 = rules[rSub.SubRule2]
            if rSub == op2:
                val2 = r.Value
                break
        if val2 == None: 
            val2 = chr(v)
            v += 1
            evalRight = True
    elif isinstance(op2, Terminal):
        for r in rules.values():
            if r.Size == op2.Size:
                val2 = r.Value
                break
        if val2 == None:
            val2 = chr(v)
            v += 1
            rules[val2] = op1
    else: val2 = op2
    op.SubRule1 = val1
    op.SubRule2 = val2
    rules[op.Value] = op # rules += [op]
    if evalLeft:
        op1.Value = val1
        rules, v = convert(op1, rules, v)
    if evalRight:
        op2.Value = val2
        rules, v = convert(op2, rules, v)
    return rules, v

# example
eq = Union('U', Terminal(1), Union(Product(Terminal(1), 'U'), \
                                   Product(Product(Terminal(1), 'U'), 'U')))
# eq = Union('X', Terminal(1), Product('X', 'X'))
rules = ConvertToStandardForm(eq)
for r in rules:
    if isinstance(r, Terminal): print str(r.Value) + " -> " + str(r.Size)
    else: print str(r.Value) + " -> " + str(r.SubRule1) + ", " + str(r.SubRule2)

A -> 1
C -> A, U
B -> C, D
U -> A, B
D -> C, U


In [51]:
# sort rules by valuation - example

rules = [Union('U', 'A', 'B'), Terminal('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
rules_sorted = sortRulesByValuation(rules)
for r in rules_sorted:
    print r.Value

A
D
C
B
U


In [ ]:
U = Z + (Z × U) + (U × Z) + (U × Z × U) <- non-empty triangulations of convex polygons
T(z) = z + 0.5 T(z)^2 + 0.5 T(z^2) <- otter trees
E = Z + E ◦ [{Z × Z} + {Z × Z × Z}] <- balanced 2-3 trees
B = 1 + z + (z + z * z)B <- binary strings with no consecutive zeros
T = z + T * T * T <- ternary trees

In [19]:
# unit tests, this will be expanded as we add functionality

def run_tests():
    test_binary_tree()
    test_unary_binary_tree()
    print "Done"
    return

def test(name, eq, n, ref):
    seq = SymbolicEquation(eq, n)
    for i in range(n):
        if seq[i] != ref[i]:
            print "> " + name + " failed, output = " + str(seq) + ", expected = " + str(ref)
    return
     
def test_binary_tree():
    eq = Union('X', Terminal(1), Product('X', 'X'))
    ref = [0, 1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862, 16796, 58786, 208012, 742900, \
           2674440, 9694845, 35357670, 129644790, 477638700]
    n = 20
    test("test_binary_tree", eq, n, ref)
    
def test_unary_binary_tree():
    eq = Union('X', Terminal(1), Union(Product(Terminal(1), 'X'), \
                                       Product(Product(Terminal(1), 'X'), 'X')))
    ref = [0, 1, 1, 2, 4, 9, 21, 51, 127, 323, 835, 2188, 5798, 15511, 41835, 113634, \
           310572, 853467, 2356779, 6536382]
    n = 20
    test("test_unary_binary_tree", eq, n, ref)

def test_convex_triangulation():
    return

def test_otter_tree():
    return
    
def test_balanced_tree():
    return

def test_restricted_binary_string():
    return

def test_ternary_tree():
    return

In [20]:
run_tests()

Done
